## 머신러닝 실습

### 텍스트 마이닝

#### 개념

##### 텍스트 마이닝
- 비정형 텍스트데이터에서 패턴을 찾음. 의미가 있는 정보들을 추출하고 분석하는 기법
- 데이터 마이닝, 자연어 처리, 정보 검색 등의 분야가 결합된 분석기법 이용
- 프로세스 - 텍스트전처리 > 특성 벡터화 > 머신러닝 모델 구축, 학습, 평가
- 텍스트 전처리 - 토큰화, 불용어 제거(은,는,이,가,을,를), 형태소 분석

##### 특성 벡터화 / 추출
- 컴퓨터는 글을 읽을 수 없음 -> 단어기반으로 특성 추출, 숫자형 벡터값으로 표현
- BoW, Word2ve 라이브러리(모듈) 존재

##### LDA 
- 문서에 잠재되어있는 토픽을 추론하는 확률 모델 알고리즘
- pyLDAvis 시각화 라이브러리 사용

#### 데이터 수집
- https://github.com/e9t/nsmc
- ratings.txt , ratings_test.txt, ratings_train.txt

- id(리뷰번호), document(리뷰), label(감성분류클래스 0 은 부정 1은 긍정)

In [1]:
## 필요 라이브러리 사용등록
import pandas as pd
import re

In [3]:
# 모든 모듈/라이브러리는 __vorsion__
pd.__version__

'2.2.1'

In [6]:
## pandas 버전 상세화 
pd.show_versions()


INSTALLED VERSIONS
------------------
commit                : bdc79c146c2e32f2cab629be240f01658cfb6cc2
python                : 3.12.2.final.0
python-bits           : 64
OS                    : Windows
OS-release            : 10
Version               : 10.0.19045
machine               : AMD64
processor             : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
byteorder             : little
LC_ALL                : None
LANG                  : None
LOCALE                : Korean_Korea.949

pandas                : 2.2.1
numpy                 : 1.26.4
pytz                  : 2024.1
dateutil              : 2.8.2
setuptools            : 69.1.1
pip                   : 24.0
Cython                : None
pytest                : None
hypothesis            : None
sphinx                : None
blosc                 : None
feather               : None
xlsxwriter            : None
lxml.etree            : None
html5lib              : None
pymysql               : None
psycopg2              : No

In [17]:
## 훈련용 데이터 가져오기
dfNsmcTrain= pd.read_csv('./data/ratings_train.txt', engine='python', sep= '\t', encoding='utf-8')

In [18]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [19]:
## 결측지 확인
dfNsmcTrain[dfNsmcTrain['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [22]:
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'].notnull()]

In [23]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [24]:
dfNsmcTrain['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [27]:
## Regular Expression(정규식) 으로 한글 이외의 문제는 다 제거
dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^ ㄱ-|가-힣]+', '', x))

error: bad character range ㄱ-| at position 3